In [2]:
!pip install confluent_kafka snowflake-connector-python pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.4 MB/s eta 0:00:00


In [12]:
!pip install sqlalchemy snowflake-sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 2.4 MB/s eta 0:00:00


In [1]:
from google.colab import files
files.upload()  # Selecciona los 3 archivos descargados

Saving ca.pem to ca.pem
Saving service.cert to service.cert
Saving service.key to service.key


{'ca.pem': b'-----BEGIN CERTIFICATE-----\nMIIETTCCArWgAwIBAgIUDcrT0p5evYF1oI5pJA4m+QecLuAwDQYJKoZIhvcNAQEM\nBQAwQDE+MDwGA1UEAww1MGM4N2M0ODctMzFjOS00ZjBmLTk2ODAtMmJjZjAzZmU5\nZmFhIEdFTiAxIFByb2plY3QgQ0EwHhcNMjUwMjI1MTYwNDA3WhcNMzUwMjIzMTYw\nNDA3WjBAMT4wPAYDVQQDDDUwYzg3YzQ4Ny0zMWM5LTRmMGYtOTY4MC0yYmNmMDNm\nZTlmYWEgR0VOIDEgUHJvamVjdCBDQTCCAaIwDQYJKoZIhvcNAQEBBQADggGPADCC\nAYoCggGBAMHf1S0eACE7MlWOWEPg8KC02684f7xWOnRBsJfd4ab+l69vmxcuidQH\nUEw6+hs1uRAIJ/H3fZsnUpnYHjtvuPa0vj08aYCmLqZdS+GeGCeTJcS8kYOdw/mK\nMUgqzSvvMJoDKOC2yN5zDBCKgTT+OkrUj8G7dfJ2u9DBqYGqFveD5mzFHSt32piP\nhq+8uOsdEHTpmBE43UvYxm4i8gcWkwhvAa+q45UG83ewQ+IXr1vQiT8W/rRV7AGp\nKC9PU8LyenbdPvy2oQIfrrA+5/Qqvgb89YpZt8ghCIXR5KVf9IxUqesVZnIIYQBI\n806DYGS3sjEpW258PmOiN/ZAjmKHCNd0HivsxBWmJ8TXu2LZkeHKPccCNTGzR+dX\nTmJY5JBna29ucWs0bpBSCVHcvPWd23Hbag88OLf9uQoM4BZmUFizMvgvbXTro2P/\nNGa7UEa3Pv3dQBAuaSR5Eoia8GfFPjDkvSwTWYl7i3msVLu3UViupi1N3VpxXUh3\n+RcLb5JSIwIDAQABoz8wPTAdBgNVHQ4EFgQUDnxJgrdN2sFHvd+CPzDq5QgWHXYw\nDwYDVR0TBAgwBgEB/wIBADALBgNVHQ8EBA

In [13]:
# %% [Celda 2] - Configuración y conexión
from sqlalchemy import create_engine
import urllib
import pandas as pd

# Configuración (usa variables o secrets)
config = {
    'user': 'KENDERAG',
    'password': 'Hp7g33snkm64alqxlixn',
    'account': 'NDVNNHP-ZM24191',
    'warehouse': 'COMPUTE_WH',
    'database': 'INV_DB',
    'schema': 'PUBLIC'
}

# Codificar contraseña para URL
password_encoded = urllib.parse.quote_plus(config['password'])

# Cadena de conexión para SQLAlchemy
connection_string = (
    f"snowflake://{config['user']}:{password_encoded}@"
    f"{config['account']}/{config['database']}/"
    f"{config['schema']}?warehouse={config['warehouse']}"
)

# Crear engine
engine = create_engine(connection_string)
print("✅ Conexión establecida con SQLAlchemy")

✅ Conexión establecida con SQLAlchemy


In [15]:
# %% [Celda 3] - Creación de vistas (con SQLAlchemy text())
from sqlalchemy import text

def create_views():
    with engine.connect() as conn:
        try:
            # Vista 1: Alertas de inventario
            conn.execute(text("""
            CREATE OR REPLACE VIEW INVENTORY_ALERTS AS
            SELECT
                PRODUCT_ID, PRODUCT_NAME, STORE_ID,
                CURRENT_STOCK, DAILY_SALES_AVG, STOCK_STATUS,
                CASE
                    WHEN DAILY_SALES_AVG > 0 THEN FLOOR(CURRENT_STOCK / DAILY_SALES_AVG)
                    ELSE NULL
                END AS DAYS_UNTIL_OUT_OF_STOCK,
                LAST_RESTOCK_DATE
            FROM TECH_PRODUCTS_STREAM
            """))

            # Vista 2: Análisis regional
            conn.execute(text("""
            CREATE OR REPLACE VIEW REGIONAL_DEMAND AS
            SELECT
                REGION, STORE_ID,
                AVG(DAILY_SALES_AVG) AS AVG_SALES,
                SUM(CURRENT_STOCK) AS TOTAL_STOCK,
                SUM(CURRENT_STOCK) / NULLIF(AVG(DAILY_SALES_AVG), 0) AS COVERAGE_DAYS
            FROM TECH_PRODUCTS_STREAM
            GROUP BY REGION, STORE_ID
            """))
            conn.commit()  # Confirmar cambios
            print("✅ Vistas creadas correctamente")
        except Exception as e:
            print(f"⛔ Error: {str(e)}")

create_views()

✅ Vistas creadas correctamente


In [16]:
# %% [Celda 4] - Consulta 1: Productos en riesgo crítico
query_risk = """
SELECT
    PRODUCT_ID, PRODUCT_NAME, STORE_ID,
    CURRENT_STOCK, DAILY_SALES_AVG,
    DAYS_UNTIL_OUT_OF_STOCK
FROM INVENTORY_ALERTS
WHERE
    STOCK_STATUS = 'RISK'
    AND DAYS_UNTIL_OUT_OF_STOCK < 3
ORDER BY DAYS_UNTIL_OUT_OF_STOCK ASC
"""

df_risk = pd.read_sql(query_risk, engine)
print("🔴 Productos en riesgo crítico (<3 días de stock):")
display(df_risk.head())

🔴 Productos en riesgo crítico (<3 días de stock):


,product_id,product_name,store_id,current_stock,daily_sales_avg,days_until_out_of_stock
0,P00022,Sony Tablet Up,STORE_NORTH_1,8,22,0
1,P00010,Sony Tablet These,STORE_NORTH_1,6,19,0
2,P00030,Xiaomi Headphones Now,STORE_NORTH_4,16,9,1
3,P00022,Sony Tablet Together,STORE_EAST_4,26,15,1
4,P00001,Xiaomi Laptop Girl,STORE_SOUTH_4,18,15,1


In [17]:
# %% [Celda 5] - Consulta 2: Exceso de stock por región
query_excess = """
SELECT
    REGION, STORE_ID,
    TOTAL_STOCK, AVG_SALES,
    COVERAGE_DAYS
FROM REGIONAL_DEMAND
WHERE COVERAGE_DAYS > 30
ORDER BY COVERAGE_DAYS DESC
"""

df_excess = pd.read_sql(query_excess, engine)
print("\n📦 Regiones con exceso de inventario:")
display(df_excess.head())


📦 Regiones con exceso de inventario:


,region,store_id,total_stock,avg_sales,coverage_days
0,SOUTH,STORE_SOUTH_2,1940,9.642857,201.185188
1,WEST,STORE_WEST_4,1971,10.692308,184.338124
2,NORTH,STORE_NORTH_2,2553,14.785714,172.666670
3,WEST,STORE_WEST_3,2099,12.916667,162.503222
4,SOUTH,STORE_SOUTH_3,1724,11.818182,145.876921


In [18]:
# %% [Celda 6] - Consulta 3: Reordenamiento por marca
query_reorder = """
SELECT
    PRODUCT_ID, PRODUCT_NAME, BRAND,
    STORE_ID, CURRENT_STOCK
FROM TECH_PRODUCTS_STREAM
WHERE
    STOCK_STATUS = 'RISK'
    AND BRAND = 'Samsung'
ORDER BY LAST_RESTOCK_DATE
"""

df_reorder = pd.read_sql(query_reorder, engine)
print("\n🔄 Productos Samsung para reordenar:")
display(df_reorder.head())


🔄 Productos Samsung para reordenar:


,product_id,product_name,brand,store_id,current_stock
0,P00021,Samsung Tablet Population,Samsung,STORE_NORTH_1,23
1,P00026,Samsung Headphones Positive,Samsung,STORE_EAST_3,47
